In [33]:
import numpy as np
import json
import itertools
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go
from scipy.integrate import quad

init_notebook_mode(connected=True)

data={"Frames":[]};


a = 0.75;
shape = "circle" 
perm = 4*np.pi*10e-7 
current = 2.; 
P = np.array([0,0,1]).tolist();
phi_scal = 0;
B = np.empty((0, 3))
iterator = 0.2 * np.pi; 

uu = np.linspace(0, 2 * np.pi, 50)
xc = a*np.cos(uu); 
yc = a*np.sin(uu); 
zc = np.zeros_like(xc); 


def calc_this(shape,phi_scal,P,a):
    if shape == "circle":
        dI= np.array([-a*np.sin(phi_scal),a*np.cos(phi_scal),0])
        r_vec = np.array([P[0]-a*np.cos(phi_scal),P[1]-a*np.sin(phi_scal),P[2]])
        
    return dI,r_vec 

def Biot_equation (current,perm,dI,r_vec):
    xprod = np.cross(dI,r_vec)
    dB = (current/(4*np.pi*(np.linalg.norm(r_vec)**3)))*xprod #should multiply current by perm 
    return dB


def calc_integrand(phi):
    a = 0.75;
    dI= np.array([-a*np.sin(phi),a*np.cos(phi),0])
    rvec = np.array([P[0]-a*np.cos(phi),P[1]-a*np.sin(phi),P[2]])
    result =(current/(4*np.pi*(np.linalg.norm(r_vec)**3)))* np.cross(dI,rvec)
    return result[i]



In [34]:

for ang in range(0,11):
    
    dI,r_vec = calc_this("circle",(phi_scal+(ang*iterator)),P,a)
    dB= 2*Biot_equation(current,perm,dI,r_vec) #value of 18 is heuristic for diagram 
    
    for j in range(3):
        i=j
        zz = quad(calc_integrand,0,2*np.pi)[0]
        B = np.append(B,zz,axis=None) #value of 2.5 is heuristic for diagram 

    Currenttrace = go.Scatter3d(name = "dI" + str(ang),
                                x = [a*np.cos((phi_scal+(ang*iterator))),dI[0]+a*np.cos((phi_scal+(ang*iterator)))], 
                                y = [a*np.sin((phi_scal+(ang*iterator))),dI[1]+a*np.sin((phi_scal+(ang*iterator)))], 
                                z = [0,dI[2]], 
                                line = dict(width = 6,color = 'rgb(250,20,0)'),
                                marker = dict(size=[0,10],color= 'rgb(250,20,0)',symbol= 'diamond',opacity=1)
                               )   


    Magtrace = go.Scatter3d(name = "dB" +  str(ang),
                            x = [P[0],0+dB[0]], 
                            y = [P[1],0+dB[1]], 
                            z = [P[2],1+dB[2]],
                            line = dict(width = 6, color = 'rgb(50,100,200)'),
                            marker = dict(size=[0,10],color= 'rgb(50,100,200)',symbol= 'diamond',opacity=1),
                           );

    Rtrace = go.Scatter3d(name = "R" + str(ang),
                          x = [a*np.cos((phi_scal+(ang*iterator))),r_vec[0]+a*np.cos((phi_scal+(ang*iterator)))],
                          y = [a*np.sin((phi_scal+(ang*iterator))),r_vec[1]+a*np.sin((phi_scal+(ang*iterator)))],
                          z = [0,r_vec[2]], 
                          line = dict(width = 6, color = 'rgb(0,220,0)'),
                          marker = dict(size=[0,10],color= 'rgb(0,220,0)',symbol= 'diamond',opacity=1),
                         );


    Btrace = go.Scatter3d(name = "B field" + str(ang) ,
                          x = [P[0],P[0]+B[0]], 
                          y = [P[1],P[1]+B[1]], 
                          z = [P[2],P[2]+B[2]],
                          line = dict(width = 6, color = 'rgb(255,160,75)'),
                          marker = dict(size=[0,10],color= 'rgb(255,160,122)',symbol= 'diamond',opacity=1)
                         );

    Circletrace = go.Scatter3d(name = "circle" + str(ang), 
                               x =  xc.tolist(),
                               y= yc.tolist(),
                               z = zc.tolist(),
                               mode = "markers",
                               marker = dict(symbol="circle",size=2.5,opacity=1,color= 'rgb(0,0,0)'),
                              );

    Pointtrace = go.Scatter3d(name = 'Point' + str(ang) ,
                              x = [P[0]],
                              y= [P[1]],
                              z=[P[2]],
                              marker = dict(size=4, color= 'rgb(214,11,8)')
                             )


    result= [Currenttrace,Magtrace,Rtrace,Btrace,Circletrace,Pointtrace]
    
    data['Frames'].append(result)
    
    print (phi_scal+(ang*iterator))

#Creating the JSON 
with open('BiotAnimate.JSON','w') as outfile:
    json.dump(data, outfile)

0.0
0.6283185307179586
1.2566370614359172
1.8849555921538759
2.5132741228718345
3.141592653589793
3.7699111843077517
4.39822971502571
5.026548245743669
5.654866776461628
6.283185307179586


In [3]:
#testing whether it works with JSON 

with open('BiotAnimate.JSON') as json_data:
    plot_data = json.load(json_data)
    print(plot_data)
    
    layout = {
    'autosize': True,
    'width': 700, 'height': 700,
    'showlegend':False,
    'scene': {
        'aspectmode': 'cube',
        'xaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'yaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'zaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'camera': {
            'up': {'x': 0, 'y': 1, 'z': 0} 
        }
    }
}

newdata = plot_data['5'];

fig = go.Figure(data=newdata[0], layout=layout)
iplot(fig)



{'Frames': [[{'type': 'scatter3d', 'name': 'dI1', 'x': [0.35, 0.3500000000000001], 'y': [-8.572527594031472e-17, 0.34999999999999987], 'z': [0, 0.0], 'line': {'width': 6, 'color': 'rgb(250,20,0)'}, 'marker': {'size': [0, 15], 'color': 'rgb(250,20,0)', 'symbol': 'diamond', 'opacity': 1}}, {'type': 'scatter3d', 'name': 'dB1', 'x': [0, 0.09367822369351479], 'y': [0, -2.2944547359214525e-17], 'z': [1, 1.0327873782927302], 'line': {'width': 6, 'color': 'rgb(50,100,200)'}, 'marker': {'size': [0, 15], 'color': 'rgb(50,100,200)', 'symbol': 'diamond', 'opacity': 1}}, {'type': 'scatter3d', 'name': 'R1', 'x': [0.35, 0.0], 'y': [-8.572527594031472e-17, 0.0], 'z': [0, 1.0], 'line': {'width': 6, 'color': 'rgb(0,220,0)'}, 'marker': {'size': [0, 15], 'color': 'rgb(0,220,0)', 'symbol': 'diamond', 'opacity': 1}}, {'type': 'scatter3d', 'name': 'B field1', 'x': [0, 1.0899589056276974e-17], 'y': [0, 1.2756600398487719e-17], 'z': [1, 1.2102134423977766], 'line': {'width': 6, 'color': 'rgb(255,160,75)'}, 'ma

KeyError: '5'

In [42]:
type(data["Frames"])

list

In [41]:
data={"Frames":[[2],[3],[4],[],[]]};

data["Frames"][4]


[]